In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [3]:
filepath = '/home/jupyter-user5/Camda24_resistance/DataSets/ResistanceCiprofloxacinStrict.tsv.gz'
df1 = pd.read_csv(filepath, sep='\t', compression='gzip')

/tmp/ipykernel_3521216/3716155289.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(filepath, sep='\t', compression='gzip')


In [4]:
df1.shape

(3881, 880)

In [5]:
df1 = df1.drop('accession', axis=1)
df1.head()

,genus,species,phenotype,mic,3005053,3000830,3003838,3000508,3003890,3000491,...,3007751-D87Y,3003926-D87Y,3003709-G46S,3004851-A39T,3004832-A501P,3003381-R20H,3003926-S83I,3003381-G121D,3004832-T483S,3004832-A311V
0,Campylobacter,jejuni,Susceptible,0.12,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Campylobacter,jejuni,Susceptible,0.06,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Campylobacter,jejuni,Susceptible,0.06,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Campylobacter,jejuni,Susceptible,0.06,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Campylobacter,jejuni,Susceptible,0.06,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
def train_multioutput_model(df):
    """
    Función para entrenar un modelo multi-output usando RandomForest, SVM, y KNN.
    Args:
        df: DataFrame con características (features) y etiquetas (labels) multioutput.
        
    Returns:
        models: Diccionario con los modelos entrenados y los F1 scores para cada salida.
    """
    
    # 1. Separar características y etiquetas
    X = df.drop(columns=['phenotype', 'mic'])  # Características
    y = df[['mic']]  # Etiquetas (multioutput)

    # 2. Manejar valores NaN en las etiquetas
    y = y.fillna(method='ffill')  # Llenar NaN con el valor anterior (puedes cambiar el método según tu preferencia)
    
    # También podrías usar y.dropna() para eliminar las filas con NaN en y

    # 3. Preprocesamiento
    binary_cols = [col for col in X.columns if set(X[col].unique()) <= {0, 1}]
    multiclass_cols = [col for col in X.columns if len(X[col].unique()) > 2]

    preprocessor = ColumnTransformer(
        transformers=[
            ('binary', 'passthrough', binary_cols),
            ('multiclass', OneHotEncoder(), multiclass_cols)
        ], remainder='passthrough'
    )

    # 4. Definir los modelos
    models = {}
    classifiers = {
        'Random Forest': RandomForestClassifier(random_state=42),
        'SVM': SVC(),
        'KNN': KNeighborsClassifier()
    }

    # 5. Entrenar cada modelo
    for name, classifier in classifiers.items():
        model_pipeline = Pipeline(steps=[
            ('preprocessing', preprocessor),
            ('classifier', classifier)
        ])

        # 6. Dividir los datos en entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Entrenar el modelo
        model_pipeline.fit(X_train, y_train)

        # Hacer predicciones y calcular el F1 score para cada salida
        y_pred = model_pipeline.predict(X_test)
        f1_scores = []
        for i in range(y_test.shape[1]):  # Para cada salida
            f1 = f1_score(y_test.iloc[:, i], y_pred[:, i], average='weighted')
            f1_scores.append(f1)

        # Guardar el modelo y los F1 scores por salida
        models[name] = {
            'model': model_pipeline,
            'f1_scores': f1_scores  # Lista con F1 score de cada salida
        }
    
    return models


In [11]:
# Entrenar los modelos
models = train_multioutput_model(df1)

# Imprimir los resultados
for name, info in models.items():
    print(f'Model: {name}')
    print(f'F1 scores por salida: {info["f1_scores"]}')


/tmp/ipykernel_3521216/4112049522.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  y = y.fillna(method='ffill')  # Llenar NaN con el valor anterior (puedes cambiar el método según tu preferencia)
/opt/conda/envs/umap/lib/python3.9/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.